In [1]:
import pandas as pd

from ortools.linear_solver import pywraplp
from ortools.init import pywrapinit

In [2]:
df_nutrients = pd.read_csv("rdi.csv")

In [3]:
df_nutrients

,Energy (kcal/d),Protein (g/d),Carbohydrate (g/d),Fat (g/d),Fibres (g/d),LA (g/d),ALA (g/d),EPA (g/d),DHA (g/d),Calcium (mg/d),...,Vitamin K1 (µg/d),Vitamin C (mg/d),Vitamin B1 or Thiamin (mg/d),Vitamin B2 or Riboflavin (mg/d),Vitamin B3 or Niacin (mg/d),Vitamin B5 or Pantothenic acid (mg/d),Vitamin B6 (mg/d),Vitamin B9 or Folate (µg/d),Vitamin B12 (µg/d),Vitamin A (µg/d)
0,2500,96.0,281.25,97.222222,30.0,11.111111,1.388889,0.125,0.125,1000.0,...,70.0,110.0,1.0467,1.6,16.7472,5.0,1.7,330.0,4.0,750.0


In [4]:
nutrients = [(key,value) for key,value in df_nutrients.to_dict("records")[0].items()]
nutrients

[('Energy (kcal/d)', 2500),
 ('Protein (g/d)', 96.0),
 ('Carbohydrate (g/d)', 281.25),
 ('Fat (g/d)', 97.22222222222224),
 ('Fibres (g/d)', 30.0),
 ('LA (g/d)', 11.11111111111111),
 ('ALA (g/d)', 1.3888888888888888),
 ('EPA (g/d)', 0.125),
 ('DHA (g/d)', 0.125),
 ('Calcium (mg/d)', 1000.0),
 ('Chloride (mg/d)', 3100.0),
 ('Copper (mg/d)', 1.6),
 ('Iron (mg/d)', 11.0),
 ('Iodine (µg/d)', 150.0),
 ('Magnesium (mg/d)', 350.0),
 ('Manganese (mg/d)', 3.0),
 ('Phosphorus (mg/d)', 550.0),
 ('Potassium (mg/d)', 3500.0),
 ('Selenium (µg/d)', 70.0),
 ('Sodium (mg/d)', 2000.0),
 ('Zinc (mg/d)', 10.0),
 ('Vitamin D (µg/d)', 15.0),
 ('Vitamin E (mg/d)', 13.0),
 ('Vitamin K1 (µg/d)', 70.0),
 ('Vitamin C (mg/d)', 110.0),
 ('Vitamin B1 or Thiamin (mg/d)', 1.0467000000000002),
 ('Vitamin B2 or Riboflavin (mg/d)', 1.6),
 ('Vitamin B3 or Niacin (mg/d)', 16.747200000000003),
 ('Vitamin B5 or Pantothenic acid (mg/d)', 5.0),
 ('Vitamin B6 (mg/d)', 1.7),
 ('Vitamin B9 or Folate (µg/d)', 330.0),
 ('Vitamin B1

In [5]:
df_foods=pd.read_csv("ciqual_2020.csv")

In [6]:
# https://stackoverflow.com/questions/11350770/filter-pandas-dataframe-by-substring-criteria
def filter_rows_by_substrings(df, col, substrings):
    return df[~df[col].str.contains('|'.join(substrings))]


# https://stackoverflow.com/questions/18172851/deleting-dataframe-row-in-pandas-based-on-column-value
def filter_rows_by_values(df, col, values):
    return df[~df[col].isin(values)]

In [7]:
# remove certain foods

remove_foods = ["Egg. powd", "Milk. powder. semi-skimmed", 
                "Decaffeinated coffee. powder. instan",
                "Decaffeinated not instant coffee. without sugar. ready-to-drink",
                "Espresso coffee. not instant coffee. without sugar. ready-to-drink",
                "Not instant coffee. without sugar. ready-to-drink",
                "Tea. brewed. without sug",
                "Royal jelly", 
                "Cocoa powder for baby beverag", 
                "Egg white. powd", 
                "Milk. powder. skimmed", 
                "Instant cereal (powder to be reconstituted) for baby from 4/6 month",
                "Milk. powder. whol",
                "Instant cereal (powder to be reconstituted) for baby from 6 month",
                "Egg yolk. powd", 
                "Gelatine. dried", 
                "Baby milk. first age. powd",
                "Baby milk. second age. powd",
                "Soya flou", 
                "Sea belt (Saccharina latissima). dried or dehydrated", 
                "Veal stock for sauce and cooking. dehydrated", 
                "Broth. stock or bouillon. meat and vegetables. with fat. dehydrated", 
                "Broth. stock or bouillon. meat and vegetables. defatted. dehydrated",
                "Broth. stock or bouillon. beef. dehydrated",
                "Madeira wine aspic. dehydrated", 
                "Nutritional y", 
                "Chewing gum. without sug", 
                "Chewing gum. sugar level unknown (average)",
                "Baking powder or raising agen", "Prepared mixed meat/fish canned. salad", "Stevia sweeten"]

df_foods_filtered = filter_rows_by_values(df_foods, "Name", remove_foods)

In [8]:
from collections import Counter

Counter(" ".join(remove_foods).replace(".", "").replace("(","").replace(")","").split()).most_common()

[('powder', 8),
 ('dehydrated', 6),
 ('powd', 5),
 ('coffee', 5),
 ('without', 5),
 ('or', 5),
 ('sugar', 4),
 ('for', 4),
 ('stock', 4),
 ('Egg', 3),
 ('Milk', 3),
 ('instant', 3),
 ('ready-to-drink', 3),
 ('baby', 3),
 ('and', 3),
 ('Broth', 3),
 ('bouillon', 3),
 ('Decaffeinated', 2),
 ('not', 2),
 ('sug', 2),
 ('Instant', 2),
 ('cereal', 2),
 ('to', 2),
 ('be', 2),
 ('reconstituted', 2),
 ('from', 2),
 ('month', 2),
 ('dried', 2),
 ('Baby', 2),
 ('milk', 2),
 ('age', 2),
 ('meat', 2),
 ('vegetables', 2),
 ('Chewing', 2),
 ('gum', 2),
 ('semi-skimmed', 1),
 ('instan', 1),
 ('Espresso', 1),
 ('Not', 1),
 ('Tea', 1),
 ('brewed', 1),
 ('Royal', 1),
 ('jelly', 1),
 ('Cocoa', 1),
 ('beverag', 1),
 ('white', 1),
 ('skimmed', 1),
 ('4/6', 1),
 ('whol', 1),
 ('6', 1),
 ('yolk', 1),
 ('Gelatine', 1),
 ('first', 1),
 ('second', 1),
 ('Soya', 1),
 ('flou', 1),
 ('Sea', 1),
 ('belt', 1),
 ('Saccharina', 1),
 ('latissima', 1),
 ('Veal', 1),
 ('sauce', 1),
 ('cooking', 1),
 ('with', 1),
 ('fat', 

In [9]:
remove_foods2 = ["Breakfast cereals. diet. plain or with honey. fortified with vitamins and chemical elemen", 
                 "Breakfast cereals. diet. with fruits. fortified with vitamins and chemical elemen",
                 "Breakfast cereals. chocolate wheat grain flakes. fortified with vitamins and chemical elemen",
                 "Baker's yeast. dehydrated", 
                 "Malted cocoa or chocolate powder for beverage. with sug",
                 "Cocoa or chocolate powder. for beverages. with sugar. fortified with vitamins and chemical elemen",
                 "Breakfast cereals. puffed/popped rice. fortified with vitamins and chemical elemen",
                 "Breakfast cereals. diet. with chocolate. fortified with vitamins and chemical elemen", 
                 "Onion. dried", 
                 "Breakfast cereals. popped or puffed wheat grain. with honey or caramel. fortified with vitamins and chemical elemen", 
                 "Poppy. seed", 
                 "Breakfast cereals. chocolate puffed/popped rice. fortified with vitamins and chemical elemen", 
                 "Cereal bar with chocolate. fortified with vitamins and mineral",
                 "Breakfast cereals. puffed/popped corn. with honey. fortified with vitamins and chemical elemen",
                 "Milk chocolate b", 
                 "Salty snacks. crackers. garnished or filled with ch", 
                 "Breakfast cereals. diet. with chocolate. not fortified", 
                 "Breakfast cereals. diet. with fruits. not fortified"
                 "Milky cereal breakfast bar. with chocolate or not. fortified with vitamins and mineral"
                 "Biscuit (cookie). snack with fruits filling", 
                 "Milk chocolate bar. filled with pralin", 
                 "Chocolate cereal b", 
                 "Breakfast cereals. puffed/popped corn. with honey (not fortified with vitamins and chemical elements)",
                 "Breakfast cereals. diet. with dried fruits. fortified with vitamins and chemical elemen", 
                 "Breakfast cereals. diet. with fruits. not fortified",
                 "Muesli. crunchy. with chocolate. with or without fruits. fortified with vitamins and chemical elemen",
                 "Breakfast cereals. mix of puffed or extruded cereals. fortified with vitamins and chemical elemen",
                 "Breakfast cereals. sweet. fortified with vitamins and chemical elements (average)",
                 "Breakfast cereals. with chocolate. not filled. fortified with vitamins and chemical elemen",
                 "Breakfast cereals. filled with chocolate or chocolate-hazelnuts. fortified with vitamins and chemical elemen",
                 "Breakfast cereals. sweet (average)",
                 "Muesli. crunchy. with fruits and/or dried fruits. grains (not fortified with vitamins and chemical elements)",
                 "Cereal bar with fruit. fortified with vitamins and mineral", 
                 "Breakfast cereals. puffed/popped cereals. wholemeal. fortified with vitamins and chemical elemen", 
                 "Breakfast cereals. with chocolate. not filled (not fortified with vitamins and chemical elements)",
                 "Breakfast cereals. wheat flakes with walnuts. hazelnuts or almonds. fortified with vitamins and chemical elemen",
                 "Grissini or bread stick", "Breakfast cereals. sweet.  not fortified with vitamins and chemical elements (average)", 
                 "Salty snacks. crackers. plain", 
                 "Potato flakes. dehydrated. plain", "Milky cereal breakfast bar. with chocolate or not. fortified with vitamins and mineral",
                 "Breakfast cereals. corn flakes. plain. fortified with vitamins and chemical elemen", 
                 "Breakfast cereals. corn flakes. sugar iced. fortified with vitamins and chemical elemen",
                 "Chocolate bar with biscui", "Chocolate confectionery. sugar coated", 
                 "Toblerone milk cholcolate bar w noug", 
                 "Wafer biscuit. plain or vanilla flavoured. prepacked",
                 "Chocolate confectionery or bar. with dairy filling", 
                 "Filo or phyllo pastry sheets. baked. without f", 
                 "Breakfast cereals. diet. plain. not fortified", 
                 "Biscuit for baby", "Breakfast cereals. chocolate puffed/popped wheat grain. fortified with vitamins and chemical elemen",
                 "Muesli. crunchy. with fruits or dried fruits. fortified with vitamins and chemical elemen",
                 "Breakfast cereals. filled with a filling other than chocolate. fortified with vitamins and chemical elemen",
                 "Cocoa or chocolate powder. for beverages. with sugar. fortified with vitamin",
                 "Breakfast cereals. filled with chocolate or chocolate-hazelnu", 
                 "Potato. crisps. plain or flavoured", 
                 "Potato crisps. \"à l'ancienne\" (old-fashioned style)", 
                 "Muesli (average)", "Muesli. flakes. fortified with vitamins and chemical elements (average food)", 
                 "Parsley. dried", "Saffron", 
                 "Breakfast cereals. rich in fibre. with or without fruits. fortified with vitamins and chemical elemen", 
                 "Cumin. seed", 
                 "Potato flakes. dehydrated. with milk or cream", 
                 "Marjoram. dried", 
                 "Provence herbs. dried", "Thyme. dried", "Basil. dried", "Shiitake mushroom. dried", "Cereal bar. low calori", "Breakfast cereals. rich in fibre. with chocolate. fortified with vitamins and chemical elemen",
                 "Molokhia powder (dried jute leaves)", "Crispbread. extruded and grilled", 
                 "Savory. dried", 
                 "Sea lettuce (Enteromorpha sp.). dried or dehydrated", 
                 "Laver (Porphyra sp.). dried or dehydrated", 
                 "Bay. leav", 
                 "Mint. dried", "Sage. dried", "Sugar and chocolate coated peanu", "North Atlantic rockweed (Ascophyllum nodosum). dried or dehydrated", 
                 "Breakfast cereals. chocolate wheat grain flakes (not fortified with vitamins and chemical elements)", 
                 "Puffed salty snacks. made from maize/corn. without peanu", 
                 "Mix of chicory and coffee. powder. instan", "Potato crisps and related. reduced f", 
                 "Chocolate b", 
                 "Dulse (Palmaria palmata). dried or dehydrated", 
                 "Breakfast cereals. chocolate puffed/popped rice (not fortified with vitamins and chemical elements)", 
                 "Biscuit (cookie). thin. w almond", 
                 "Gracilaria seaweeds (Gracilaria verrucosa). dried or dehydrated", 
                 "Mix of 4 spi", "Salty snacks. crackers. reduced f", "Coated chocolate bar without biscui", "Oregano. dried", "Rosemary. dried", 
                 "Kombu or Japanese kelp (Laminaria japonica). dried or dehydrated", 
                 "Sea thong (Himanthalia elongata). dried or dehydrated", 
                 "Sea lettuce (ulva sp.). dried or dehydrated)", 
                 "Wakame (Undaria pinnatifida). dried or dehydrated", 
                 "Fennel. seed", "Salty snack", "Corn chips or tortilla chip", 
                 "Coffee. powder. instan", 
                 "Cayenne pepp", 
                 "Tangle (Laminaria digitata). dried or dehydrated", 
                 "Soy lecithin", 
                 "Puffed salty snacks. made from potato", "Coffee. ground", 
                 "Toothed wrack or bladder wrack (Fucus serratus et vesiculosus). dried or dehydrated", 
                 "Cocoa or chocolate powder. for beverages. with sug", 
                 "Cocoa powder. without sugar. powder. instan", 
                 "Breakfast biscuit (cookie) with cereals. fortified with vitamins and chemical elemen", 
                 "Prawn crack", 
                 "Pop-corn or popped maize. with caramel", 
                 "Wheat crack", 
                 "Crouton with garlic. herbs or onions. prepacked", 
                 "Black pepper. powd", 
                 "Muesli. flakes (Bircher-style). with fruits or dried fruits. fortified with vitamins and chemical elemen", 
                 "Florentine biscuit (chocolate sweet biscuit (cookie) with almonds)", 
                 "Coconut bar. with chocolate coating", "Cereal bar w frui", 
                 "Chewing gum. with sug", 
                 "Chocolate bar with dried frui", 
                 "Breakfast cereals. corn flakes. sugar iced (not fortified with vitamins and chemical elements)", "Puffed rice textured bread. wholemeal"]

df_foods_filtered2 = filter_rows_by_values(df_foods_filtered, "Name", remove_foods2)

In [10]:
from collections import Counter

Counter(" ".join(remove_foods2).replace(".", "").replace("(","").replace(")","").split()).most_common()
    

[('with', 83),
 ('fortified', 42),
 ('and', 42),
 ('or', 40),
 ('vitamins', 38),
 ('cereals', 36),
 ('Breakfast', 34),
 ('chemical', 34),
 ('dried', 29),
 ('chocolate', 27),
 ('elemen', 25),
 ('not', 15),
 ('dehydrated', 14),
 ('fruits', 13),
 ('bar', 13),
 ('flakes', 10),
 ('powder', 9),
 ('elements', 9),
 ('diet', 8),
 ('plain', 7),
 ('puffed/popped', 7),
 ('filled', 7),
 ('sugar', 6),
 ('Chocolate', 6),
 ('Muesli', 6),
 ('without', 6),
 ('wheat', 5),
 ('for', 5),
 ('corn', 5),
 ('snacks', 5),
 ('average', 5),
 ('Potato', 5),
 ('honey', 4),
 ('grain', 4),
 ('Cocoa', 4),
 ('rice', 4),
 ('Cereal', 4),
 ('Salty', 4),
 ('cookie', 4),
 ('sweet', 4),
 ('biscuit', 4),
 ('sug', 3),
 ('beverages', 3),
 ('seed', 3),
 ('mineral', 3),
 ('b', 3),
 ('crackers', 3),
 ('cereal', 3),
 ('filling', 3),
 ('crunchy', 3),
 ('of', 3),
 ('w', 3),
 ('f', 3),
 ('crisps', 3),
 ('Sea', 3),
 ('sp', 3),
 ('Puffed', 3),
 ('instan', 3),
 ('popped', 2),
 ('puffed', 2),
 ('caramel', 2),
 ('Milk', 2),
 ('breakfast', 2

In [11]:
substrings = ["fortified", 
              "cereal", "Cereal", 
              "dried", 
              "powder", "powd", 
              "dehydrated", 
              "Muesli", 
              "snack", 
              "Biscuit", "biscuit", 
              "Chewing gum", 
              "crack", 
              "seed", 
              "Chocolate", "chocolate", "cholcolate", 
              "flou", 
              "crisps", "chip", 
              "Bay. leav",  
              "Saffron", 
              "Coffee", 
              "Soy lecithin", 
              "pastry", "Pastry", 
              "Cayenne pepp", "Pop-corn or popped maize. with caramel", 
              "Crispbread. extruded and grilled", 
              "Spice", 
              "Crouton", 
              "Royal jelly", 
              "oil", 
              "Paprik", 
              "Almond paste or marzipan. prepacked", 
              "Clov", 
              "fat", "Palm kernel f", "Turkey f", "Duck f", "Chicken f", "Goose f", "pork f", "Shea butter", 
              "Mix", "Cocoa bu", "bread", "Raisin", "Baby", "Stevia sweeten", "pre-cooked"]

df_foods_filtered2 = filter_rows_by_substrings(df_foods, "Name", substrings)

In [12]:
commodities = list(df_foods_filtered2["Name"])

In [13]:
data = df_foods_filtered2.drop("Name", axis=1).values.tolist()

In [14]:
solver = pywraplp.Solver.CreateSolver('GLOP')

In [15]:
# Declare an array to hold our variables. 
foods = [solver.NumVar(0.0, solver.infinity(), item) for item in commodities]

print('Number of variables =', solver.NumVariables())

Number of variables = 2332


In [16]:
# Create the constraints, one per nutrient. (data = nutrients_per_100_gramm)
# gurobipy can express a lists or arrays of constraints with a nicer DSL 
# instead of the many loops necessary with OR-Tools
constraints = []
for i, nutrient in enumerate(nutrients):
    constraints.append(solver.Constraint(nutrient[1], solver.infinity(), nutrient[0]))
    for j, item in enumerate(data):
        constraints[i].SetCoefficient(foods[j], item[i])

print('Number of constraints =', solver.NumConstraints())

Number of constraints = 33


In [17]:
# Objective function: Minimize the sum of (price-normalized) foods.
objective = solver.Objective()
for i, food in enumerate(foods):
    objective.SetCoefficient(food, 1.0)
objective.SetMinimization()

In [18]:
status = solver.Solve()

# Check that the problem has an optimal solution.
if status != solver.OPTIMAL:
    print('The problem does not have an optimal solution!')
    if status == solver.FEASIBLE:
        print('A potentially suboptimal solution was found.')
    else:
        print('The solver could not solve the problem.')
        exit(1)

In [19]:
# Display the amounts (in dollars) to purchase of each food.
print('\nDaily Foods:')
for i, food in enumerate(foods):
    if food.solution_value() > 0.0:
        print('{}: {} gr'.format(commodities[i], food.solution_value()*100))
print('\nOptimal daily weight: {:.4f} gr'.format(objective.Value()*100))


Daily Foods:
Acerola. pulp. raw. sampled in the island of La Martiniqu: 3.5195038140567134 gr
Cashew nut. dry-grilled. unsalted: 30.65824535276519 gr
Soybean. whole grain: 142.14887092053908 gr
Rusk: 237.10250331083623 gr
Rusk. unsalted: 79.2460974763866 gr
Liver. lamb. raw: 0.7602913698617791 gr
Cod liver. raw: 24.53765406482579 gr
Gruyere cheese. from cow's milk: 52.766158786441075 gr
Pure sea salt. no enrichmen: 1.2135981791496493 gr
Rice bran: 14.923548092531949 gr

Optimal daily weight: 586.8765 gr


In [20]:
food_per_day=[]

nutrients_result = [0] * len(nutrients)
for i, food in enumerate(foods):
    if food.solution_value() > 0.0:
        # default value for weight optimization
        nutrient_per_goal = 100  
        food_per_day.append((commodities[i], food.solution_value() * nutrient_per_goal))
        for j, _ in enumerate(nutrients):
            nutrients_result[j] += data[i][j] * food.solution_value()

df_food_per_day = pd.DataFrame(food_per_day, columns=("food", "weight"))
df_food_per_day.sort_values(by='weight', ascending=False)

,food,weight
3,Rusk,237.102503
2,Soybean. whole grain,142.148871
4,Rusk. unsalted,79.246097
7,Gruyere cheese. from cow's milk,52.766159
1,Cashew nut. dry-grilled. unsalted,30.658245
6,Cod liver. raw,24.537654
9,Rice bran,14.923548
0,Acerola. pulp. raw. sampled in the island of L...,3.519504
8,Pure sea salt. no enrichmen,1.213598
5,Liver. lamb. raw,0.760291


In [21]:
print('\nNutrients per day:')
for i, nutrient in enumerate(nutrients):
    print('{}: {:.2f} (min {})'.format(nutrient[0], nutrients_result[i],
                                       nutrient[1]))


Nutrients per day:
Energy (kcal/d): 2500.00 (min 2500)
Protein (g/d): 104.33 (min 96.0)
Carbohydrate (g/d): 281.25 (min 281.25)
Fat (g/d): 98.73 (min 97.22222222222224)
Fibres (g/d): 34.80 (min 30.0)
LA (g/d): 18.50 (min 11.11111111111111)
ALA (g/d): 2.41 (min 1.3888888888888888)
EPA (g/d): 1.60 (min 0.125)
DHA (g/d): 1.20 (min 0.125)
Calcium (mg/d): 1000.00 (min 1000.0)
Chloride (mg/d): 3369.21 (min 3100.0)
Copper (mg/d): 3.14 (min 1.6)
Iron (mg/d): 31.65 (min 11.0)
Iodine (µg/d): 183.27 (min 150.0)
Magnesium (mg/d): 664.71 (min 350.0)
Manganese (mg/d): 7.93 (min 3.0)
Phosphorus (mg/d): 1927.31 (min 550.0)
Potassium (mg/d): 3500.00 (min 3500.0)
Selenium (µg/d): 89.81 (min 70.0)
Sodium (mg/d): 2000.00 (min 2000.0)
Zinc (mg/d): 17.32 (min 10.0)
Vitamin D (µg/d): 25.44 (min 15.0)
Vitamin E (mg/d): 13.00 (min 13.0)
Vitamin K1 (µg/d): 83.39 (min 70.0)
Vitamin C (mg/d): 110.00 (min 110.0)
Vitamin B1 or Thiamin (mg/d): 2.62 (min 1.0467000000000002)
Vitamin B2 or Riboflavin (mg/d): 1.88 (min

In [22]:
pd.set_option('display.max_columns', 500)

nutrient_per_food = {}

for i, food in enumerate(foods):
    if food.solution_value() > 0.0:      
        for j, nutrient in enumerate(nutrients):
            if food.name() in nutrient_per_food:
                nutrient_per_food[food.name()].append(data[i][j] * food.solution_value())
            else:
                nutrient_per_food[food.name()]=[data[i][j] * food.solution_value()]
                
foods_df = pd.DataFrame.from_dict(nutrient_per_food, orient='index', columns=[n[0] for n in nutrients])

for i, nutrient in enumerate(nutrients):
    foods_df[nutrient[0]]=(foods_df[nutrient[0]]/nutrients_result[i]*100).round(2)

display(foods_df)     

,Energy (kcal/d),Protein (g/d),Carbohydrate (g/d),Fat (g/d),Fibres (g/d),LA (g/d),ALA (g/d),EPA (g/d),DHA (g/d),Calcium (mg/d),Chloride (mg/d),Copper (mg/d),Iron (mg/d),Iodine (µg/d),Magnesium (mg/d),Manganese (mg/d),Phosphorus (mg/d),Potassium (mg/d),Selenium (µg/d),Sodium (mg/d),Zinc (mg/d),Vitamin D (µg/d),Vitamin E (mg/d),Vitamin K1 (µg/d),Vitamin C (mg/d),Vitamin B1 or Thiamin (mg/d),Vitamin B2 or Riboflavin (mg/d),Vitamin B3 or Niacin (mg/d),Vitamin B5 or Pantothenic acid (mg/d),Vitamin B6 (mg/d),Vitamin B9 or Folate (µg/d),Vitamin B12 (µg/d),Vitamin A (µg/d)
Acerola. pulp. raw. sampled in the island of La Martiniqu,0.00,0.02,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,91.19,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
Cashew nut. dry-grilled. unsalted,7.57,5.11,2.56,15.22,5.02,14.79,1.14,0.19,0.25,1.17,0.22,26.35,5.52,0.00,12.91,6.96,8.43,5.96,6.83,0.12,9.91,0.30,1.37,11.51,0.14,4.21,0.80,1.03,6.01,2.32,1.13,0.00,0.03
Soybean. whole grain,23.82,47.01,10.51,27.64,53.10,69.48,75.98,0.00,0.00,31.27,0.00,39.83,70.52,0.47,54.10,40.53,43.22,70.67,13.93,0.21,24.21,0.00,9.29,80.12,7.75,47.23,65.63,13.75,38.66,25.30,67.17,0.00,0.20
Rusk,38.41,22.27,64.58,14.17,21.12,10.90,6.88,1.48,1.97,6.88,62.35,12.08,8.24,25.88,8.56,18.55,12.30,10.84,52.80,60.34,10.82,2.33,15.14,6.31,0.00,10.87,1.26,9.91,24.66,8.55,7.62,0.00,8.24
Rusk. unsalted,12.71,7.90,20.85,4.72,11.75,1.54,0.53,0.00,0.66,1.86,0.00,4.54,3.26,2.72,2.68,7.00,4.98,3.65,2.96,0.72,32.03,0.00,28.65,0.00,0.00,19.97,9.25,40.22,0.00,33.15,10.73,0.00,0.00
Liver. lamb. raw,0.05,0.16,0.01,0.04,0.00,0.01,0.01,0.00,0.00,0.01,0.00,1.69,0.15,0.02,0.02,0.02,0.14,0.07,0.41,0.02,0.18,0.01,0.04,0.01,0.03,0.10,1.46,0.82,0.93,0.13,0.25,18.21,0.22
Cod liver. raw,6.08,1.18,0.00,16.55,0.00,1.49,8.03,98.33,97.11,0.25,0.00,5.16,3.10,66.95,0.30,0.00,1.27,0.91,17.35,7.23,2.75,96.44,37.75,0.00,0.89,0.94,8.46,3.66,3.14,1.64,10.60,61.34,82.05
Gruyere cheese. from cow's milk,8.99,14.36,0.00,18.49,0.00,1.80,7.43,0.00,0.00,57.52,16.29,6.89,0.48,3.97,3.11,0.21,16.65,1.52,3.13,8.44,14.90,0.91,2.11,1.71,0.00,1.01,10.92,0.32,4.54,1.90,1.14,20.45,9.25
Pure sea salt. no enrichmen,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.21,21.14,0.00,0.00,0.00,0.77,0.00,0.00,0.04,0.00,22.88,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
Rice bran,2.38,1.99,1.49,3.16,9.01,0.00,0.00,0.00,0.00,0.85,0.00,3.47,8.72,0.00,17.53,26.74,13.01,6.35,2.59,0.04,5.20,0.00,5.65,0.34,0.00,15.67,2.22,30.30,22.06,27.03,1.35,0.00,0.00


In [23]:
foods_df.sum()

Energy (kcal/d)                          100.01
Protein (g/d)                            100.00
Carbohydrate (g/d)                       100.00
Fat (g/d)                                 99.99
Fibres (g/d)                             100.00
LA (g/d)                                 100.01
ALA (g/d)                                100.00
EPA (g/d)                                100.00
DHA (g/d)                                 99.99
Calcium (mg/d)                           100.02
Chloride (mg/d)                          100.00
Copper (mg/d)                            100.01
Iron (mg/d)                               99.99
Iodine (µg/d)                            100.01
Magnesium (mg/d)                          99.98
Manganese (mg/d)                         100.01
Phosphorus (mg/d)                        100.00
Potassium (mg/d)                         100.01
Selenium (µg/d)                          100.00
Sodium (mg/d)                            100.00
Zinc (mg/d)                             

In [24]:
import re
def remove_unit(column_name):
    return re.sub(r"\((.*)\)", "", column_name).strip()

In [25]:
for name in nutrient_per_food.keys():
    print(foods_df.loc[[name]].sort_values(name, axis=1, ascending=False).iloc[:, :5].to_dict("index"))

{'Acerola. pulp. raw. sampled in the island of La Martiniqu': {'Vitamin C (mg/d)': 91.19, 'Protein (g/d)': 0.02, 'Selenium (µg/d)': 0.0, 'Sodium (mg/d)': 0.0, 'Zinc (mg/d)': 0.0}}
{'Cashew nut. dry-grilled. unsalted': {'Copper (mg/d)': 26.35, 'Fat (g/d)': 15.22, 'LA (g/d)': 14.79, 'Magnesium (mg/d)': 12.91, 'Vitamin K1 (µg/d)': 11.51}}
{'Soybean. whole grain': {'Vitamin K1 (µg/d)': 80.12, 'ALA (g/d)': 75.98, 'Potassium (mg/d)': 70.67, 'Iron (mg/d)': 70.52, 'LA (g/d)': 69.48}}
{'Rusk': {'Carbohydrate (g/d)': 64.58, 'Chloride (mg/d)': 62.35, 'Sodium (mg/d)': 60.34, 'Selenium (µg/d)': 52.8, 'Energy (kcal/d)': 38.41}}
{'Rusk. unsalted': {'Vitamin B3 or Niacin (mg/d)': 40.22, 'Vitamin B6 (mg/d)': 33.15, 'Zinc (mg/d)': 32.03, 'Vitamin E (mg/d)': 28.65, 'Carbohydrate (g/d)': 20.85}}
{'Liver. lamb. raw': {'Vitamin B12 (µg/d)': 18.21, 'Copper (mg/d)': 1.69, 'Vitamin B2 or Riboflavin (mg/d)': 1.46, 'Vitamin B5 or Pantothenic acid (mg/d)': 0.93, 'Vitamin B3 or Niacin (mg/d)': 0.82}}
{'Cod liver. 

In [26]:
pd.set_option('display.max_columns', 500)

nutrient_per_food = {}

for i, food in enumerate(foods):
    if food.solution_value() > 0.0:      
        for j, nutrient in enumerate(nutrients):
            if food in nutrient_per_food:
                nutrient_per_food[food].append(data[i][j] * food.solution_value())
            else:
                nutrient_per_food[food]=[data[i][j] * food.solution_value()]
                
foods_df = pd.DataFrame.from_dict(nutrient_per_food, orient='index', columns=[n[0] for n in nutrients])

for i, nutrient in enumerate(nutrients):
    foods_df[nutrient[0]]=(foods_df[nutrient[0]]).round(2)

# foods_df.loc['total'] = foods_df.iloc[1:, :-1].sum()    
    
display(foods_df)  

,Energy (kcal/d),Protein (g/d),Carbohydrate (g/d),Fat (g/d),Fibres (g/d),LA (g/d),ALA (g/d),EPA (g/d),DHA (g/d),Calcium (mg/d),Chloride (mg/d),Copper (mg/d),Iron (mg/d),Iodine (µg/d),Magnesium (mg/d),Manganese (mg/d),Phosphorus (mg/d),Potassium (mg/d),Selenium (µg/d),Sodium (mg/d),Zinc (mg/d),Vitamin D (µg/d),Vitamin E (mg/d),Vitamin K1 (µg/d),Vitamin C (mg/d),Vitamin B1 or Thiamin (mg/d),Vitamin B2 or Riboflavin (mg/d),Vitamin B3 or Niacin (mg/d),Vitamin B5 or Pantothenic acid (mg/d),Vitamin B6 (mg/d),Vitamin B9 or Folate (µg/d),Vitamin B12 (µg/d),Vitamin A (µg/d)
Acerola. pulp. raw. sampled in the island of La Martiniqu,0.00,0.02,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,100.31,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
Cashew nut. dry-grilled. unsalted,189.16,5.33,7.20,15.02,1.75,2.73,0.03,0.00,0.00,11.65,7.33,0.83,1.75,0.00,85.84,0.55,162.49,208.48,6.13,2.45,1.72,0.08,0.18,9.60,0.15,0.11,0.02,0.17,0.30,0.05,7.85,0.00,0.48
Soybean. whole grain,595.60,49.04,29.57,27.29,18.48,12.85,1.83,0.00,0.00,312.73,0.00,1.25,22.32,0.85,359.64,3.21,832.99,2473.39,12.51,4.26,4.19,0.00,1.21,66.81,8.53,1.24,1.24,2.30,1.93,0.57,466.25,0.00,3.08
Rusk,960.27,23.24,181.62,13.99,7.35,2.02,0.17,0.02,0.02,68.76,2100.73,0.38,2.61,47.42,56.90,1.47,237.10,379.36,47.42,1206.85,1.87,0.59,1.97,5.26,0.00,0.28,0.02,1.66,1.23,0.19,52.87,0.00,125.66
Rusk. unsalted,317.78,8.24,58.64,4.66,4.09,0.29,0.01,0.00,0.01,18.62,0.00,0.14,1.03,4.99,17.83,0.55,95.89,127.59,2.65,14.34,5.55,0.00,3.72,0.00,0.00,0.52,0.17,6.74,0.00,0.74,74.49,0.00,0.00
Liver. lamb. raw,1.13,0.17,0.02,0.04,0.00,0.00,0.00,0.00,0.00,0.05,0.00,0.05,0.05,0.03,0.14,0.00,2.77,2.38,0.37,0.44,0.03,0.00,0.01,0.01,0.03,0.00,0.03,0.14,0.05,0.00,1.75,0.73,3.40
Cod liver. raw,151.89,1.23,0.00,16.34,0.00,0.27,0.19,1.57,1.17,2.45,0.00,0.16,0.98,122.69,1.96,0.00,24.54,31.90,15.58,144.53,0.48,24.54,4.91,0.00,0.98,0.02,0.16,0.61,0.16,0.04,73.61,2.45,1251.42
Gruyere cheese. from cow's milk,224.78,14.99,0.00,18.26,0.00,0.33,0.18,0.00,0.00,575.15,548.77,0.22,0.15,7.28,20.68,0.02,320.82,53.29,2.81,168.85,2.58,0.23,0.27,1.42,0.00,0.03,0.21,0.05,0.23,0.04,7.91,0.82,141.15
Pure sea salt. no enrichmen,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,2.08,712.38,0.00,0.00,0.00,5.15,0.00,0.00,1.25,0.00,457.53,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
Rice bran,59.40,2.07,4.19,3.12,3.13,0.00,0.00,0.00,0.00,8.51,0.00,0.11,2.76,0.00,116.55,2.12,250.72,222.36,2.33,0.75,0.90,0.00,0.73,0.28,0.00,0.41,0.04,5.07,1.10,0.61,9.40,0.00,0.00


In [27]:
foods_df.sum()

Energy (kcal/d)                          2500.01
Protein (g/d)                             104.33
Carbohydrate (g/d)                        281.24
Fat (g/d)                                  98.72
Fibres (g/d)                               34.80
LA (g/d)                                   18.49
ALA (g/d)                                   2.41
EPA (g/d)                                   1.59
DHA (g/d)                                   1.20
Calcium (mg/d)                           1000.00
Chloride (mg/d)                          3369.21
Copper (mg/d)                               3.14
Iron (mg/d)                                31.65
Iodine (µg/d)                             183.26
Magnesium (mg/d)                          664.69
Manganese (mg/d)                            7.92
Phosphorus (mg/d)                        1927.32
Potassium (mg/d)                         3500.00
Selenium (µg/d)                            89.80
Sodium (mg/d)                            2000.00
Zinc (mg/d)         

In [28]:
activities = solver.ComputeConstraintActivities()
o = [{'Name':c.name(), 'shadow price':c.dual_value(), 'slack (%)': (activities[i] - c.lb())/c.lb()*100} for i, c in enumerate(solver.constraints())]
print(pd.DataFrame(o).round(2))

                                     Name  shadow price  slack (%)
0                         Energy (kcal/d)          0.00       0.00
1                           Protein (g/d)          0.00       8.67
2                      Carbohydrate (g/d)          0.01       0.00
3                               Fat (g/d)          0.00       1.55
4                            Fibres (g/d)          0.00      16.00
5                                LA (g/d)          0.00      66.46
6                               ALA (g/d)          0.00      73.77
7                               EPA (g/d)          0.00    1179.71
8                               DHA (g/d)          0.00     862.15
9                          Calcium (mg/d)          0.00      -0.00
10                        Chloride (mg/d)          0.00       8.68
11                          Copper (mg/d)          0.00      96.31
12                            Iron (mg/d)          0.00     187.70
13                          Iodine (µg/d)          0.00      2

In [29]:
activities = solver.ComputeConstraintActivities()
o = [{'Name':c.name(), 'shadow price':c.dual_value(), 'slack': (activities[i] - c.lb())} for i, c in enumerate(solver.constraints())]
df_sensitivity = pd.DataFrame(o)
print(df_sensitivity.round(2))

                                     Name  shadow price    slack
0                         Energy (kcal/d)          0.00     0.00
1                           Protein (g/d)          0.00     8.33
2                      Carbohydrate (g/d)          0.01     0.00
3                               Fat (g/d)          0.00     1.50
4                            Fibres (g/d)          0.00     4.80
5                                LA (g/d)          0.00     7.38
6                               ALA (g/d)          0.00     1.02
7                               EPA (g/d)          0.00     1.47
8                               DHA (g/d)          0.00     1.08
9                          Calcium (mg/d)          0.00    -0.00
10                        Chloride (mg/d)          0.00   269.21
11                          Copper (mg/d)          0.00     1.54
12                            Iron (mg/d)          0.00    20.65
13                          Iodine (µg/d)          0.00    33.27
14                       

The constraints with a slack value of zero are the most critical for the solution. 
It these constraints are changed the solution will also change. 
There are much more critical constraint for the calories optimized diet than for the weight optimized diet. 
The higher the shadow price the more sensitive is the objective function to changes of that constraint.
So the most critical constraints are 

In [30]:
df_sensitivity.sort_values("shadow price", ascending=False).loc[df_sensitivity["slack"].round(2) == 0.0].round(2)

,Name,shadow price,slack
22,Vitamin E (mg/d),0.01,0.0
31,Vitamin B12 (µg/d),0.01,0.0
2,Carbohydrate (g/d),0.01,0.0
28,Vitamin B5 or Pantothenic acid (mg/d),0.00,-0.0
0,Energy (kcal/d),0.00,0.0
27,Vitamin B3 or Niacin (mg/d),0.00,0.0
9,Calcium (mg/d),0.00,-0.0
24,Vitamin C (mg/d),0.00,0.0
17,Potassium (mg/d),0.00,0.0
19,Sodium (mg/d),0.00,0.0
